In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

### Proving error in estimating variance of Fourier mode using finite difference SHE

This notebook will be used to outline the proof that using my finite difference scheme to obtain estimates of $\mathbb{V}\left[u_n(t)\right]$, i.e. the $n$-th Fourier mode of $u(x,t)$ has error $|\mathbb{V}\left[u_n(t)\right] - \mathbb{V}\left[\hat{u}_n(t)\right]| \sim O(\Delta x^2)$. 

I'll begin by reviewing the quantity I'm trying to derive. Fourier modes are found via the following relationships:
$$
u(x,t) = \sum_{n=0}^\infty u_n(t) \phi_n(x) \\
\phi_n(x) = \sqrt{2} \sin(n \pi x), \quad \langle \phi_n, \phi_m \rangle = \delta_{n,m} \\
u_n(t) = \langle u(x,t), \phi_n \rangle = \int_0^1 u(x,t) \phi_n(x) dx \\
u_n(t) \sim \mathcal{N}(0,\frac{1 - e^{-2\lambda_n^2t}}{2\lambda_n^2}), \quad \lambda_n = \pi^2 n^2 \\
\lim_{t \to \infty}\mathbb{V}\left[u_n\right] = \frac{1}{2\lambda_n^2} \quad (*)
$$
So the long term true variance of our quantity of interest is $(*)$.

Now let's examine what we are doing in our discrete approximation of the above operations.
At this point getting a good notation is going to be really important too. I discretise the spatial domain $[0,1]$ into $N+2$ points, equivalently $N+1$ subdivisions, each of width $\Delta x = \frac{1}{N+1}$. This means we have $N$ internal points. Each discrete spatial point is notated as $x_i$, corresponding to the $i \Delta x = \frac{i}{N+1}$ where $i \in \{0, 1, ..., N+1\}$. Similarly, we discretise our temporal grid. We have $\Delta t = \lambda \Delta x^2, \quad t_j = j \Delta t, \quad j \in \{0, 1, 2, ...\}$. We express our discretised points of u(x_i, t_j) using the following notation: $u(x_i, t_j) := u_i^j$. Finally, my finite volume derived scheme for the SHE is the following

$$
u_i^{j+1} =  u_i^j + \frac{\Delta t}{\Delta x^2} \left(u_{i+1}^j - 2u_i^j + u_{i-1}^j\right) + \sqrt{\frac{\Delta t}{\Delta x}} Z_i^j, \quad Z_i^j \sim \mathcal{N}(0, 1)\\
\text{with boundary conditions:} \quad u_0^j = u_N^j = 0, \quad u_i^0 = 0
$$

Now, what I will be doing is copying the sequence of operations I used to derive the Fourier mode in the continuous case, but this time in the discrete equivalent case, and in this way I shall be able to demonstrate the consistency error in the results. As we're now in discrete space, I'll move to describing things using vectors and matrices. A better notation now becomes $\bar{u}^j := (u_0^j, u_1^j, ..., u_{N+1}^j)^T$. The basis functions become basis vectors in the new space, as we've discretised. $(\phi_n)_i = \sqrt{2} \sin(n \pi x_i)$, $\bar{\phi}_n = \sqrt{2}(\sin(n \pi x_1), \sin(n \pi x_2), ..., \sin(n \pi x_N))^T$ In matrix vector form the finite difference scheme now becomes:

$$
\bar{u}^j = \begin{bmatrix}
u_1^j \\
u_2^j \\
\vdots \\
u_N^j
\end{bmatrix}
\in \mathbb{R}^N, \quad 
Z^j =
\begin{bmatrix}
Z_1^j \\
Z_2^j \\
\vdots \\
Z_N^j
\end{bmatrix}
\sim \mathcal{N}(0, I_N), \quad
\bar{\phi}_n = \sqrt{2}
\begin{bmatrix}
\sin\left( \frac{n \pi}{N+1} \cdot 1 \right) \\
\sin\left( \frac{n \pi}{N+1} \cdot 2 \right) \\
\vdots \\
\sin\left( \frac{n \pi}{N+1} \cdot N \right)
\end{bmatrix}
\in \mathbb{R}^N \\


\bar{u}^{j+1} = \bar{u}^j + \Delta t A\bar{u}^j + \sqrt{\frac{\Delta t}{\Delta x}} Z^j\\
\text{where } \quad A = \frac{1}{\Delta x^2} \begin{bmatrix}
-2 & 1 & & \\
1 & -2 & 1 & \\
& \ddots & \ddots & \ddots \\
& & 1 & -2
\end{bmatrix} \\
\text{Importantly, we now just consider the internal points for the above. So } i \in \{1, 2, ..., N\}.
\text{What's kind of neat now is that the Fourier decomposition becomes: }\\
\bar{u}^j = \sum_{n=1}^{N}\hat{u}_n^j \bar{\phi}_n\\
\text{i.e. we can express $\bar{u}^j$ as a combination of our $N$ basis vectors scaled by Fourier modes.}\\
\text{This is what Prof. Giles was referring to when he said I can express $u_i^j$ as a finite sum.}
$$ 

From the NSPDEs course, Süli introduced the discrete inner product to us. It was: $\left(V, W\right)_h := \sum_{i=1}^{N} h V_i W_i$.
This is the same thing we can use for obtaining our discrete Fourier modes. So, we will do the same series of operations now:

$$
\bar{u}^{j+1} = \bar{u}^j + \Delta t A\bar{u}^j + \sqrt{\frac{\Delta t}{\Delta x}} Z^j \\
\text{Orthogonality of basis functions: } (\bar{\phi}_n, \bar{\phi}_m)_h := \sum_{i=1}^{N} \Delta x \cdot \phi_n(x_i) \cdot \phi_m(x_i) = \boxed{
\begin{cases}
0, & \text{if } n \ne m \\
1, & \text{if } n = m
\end{cases}
}\\
\text{As we have this result from the theory of discrete sine transforms: }
\sum_{i=1}^{N}
\sin\left( \frac{n \pi i}{N+1} \right)
\cdot
\sin\left( \frac{m \pi i}{N+1} \right)
=
\begin{cases}
\frac{N+1}{2}, & \text{if } n = m \\
0, & \text{if } n \ne m
\end{cases}\\
\text{So let's apply that result to our discrete approximations: }
(\bar{u}^{j+1}, \bar{\phi}_n)_h = (\bar{u}^j, \bar{\phi}_n)_h + \Delta t (A \bar{u}^j, \bar{\phi}_n)_h + \sqrt{ \frac{\Delta t}{\Delta x} } (Z^j, \bar{\phi}_n)_h\\
\text{LHS: } (\bar{u}^{j+1}, \bar{\phi}_n)_h = \left(\sum_{k=1}^{N}\hat{u}_k^{j+1} \bar{\phi}_k, \bar{\phi}_n\right)_h = \hat{u}_n^{j+1}\\
\text{RHS, first term: } (\bar{u}^j, \bar{\phi}_n)_h = \hat{u}_n^{j}\\
\text{RHS, second term: } \Delta t (A \bar{u}^j, \bar{\phi}_n)_h = \Delta t \Delta x \left(A \bar{u}^j\right)^T \bar{\phi}_n = \Delta t \Delta x (\bar{u}^j)^T A^T \bar{\phi}_n = \Delta t \Delta x (\bar{u}^j)^T A \bar{\phi}_n\\
$$

Now, $A$ is a symmetric matrix, and $\bar{\phi}_n$ are actually eigenvectors
(see [Wikipedia](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors_of_the_second_derivative#Pure_Dirichlet_boundary_conditions_2)).

The eigenvalues of $A$ are given by: $-\lambda_n^{FD} = -\frac{4}{\Delta x^2} \sin^2\left( \frac{\pi n}{2(N+1)} \right)$. So $A \bar{\phi}_n = -\lambda_n^{FD} \bar{\phi}_n$. Returning to our calculations:

$$
\Delta t \Delta x (\bar{u}^j)^T A \bar{\phi}_n = \Delta t \Delta x (\bar{u}^j)^T \lambda_n \bar{\phi}_n = \Delta t \lambda_n^{FD} \left(\bar{u}^j, \bar{\phi}_n\right) = -\Delta t \lambda_n^{FD} \hat{u}_n^j\\
\text{That's neat. Final RHS term: } \sqrt{ \frac{\Delta t}{\Delta x} } (Z^j, \bar{\phi}_n)_h = \sqrt{ \frac{\Delta t}{\Delta x} } \sum_{k=1}^N \Delta x Z_k^j (\bar{\phi}^j)_k \quad \text{And this is a normal RV. Sum of normals.}
 $$

 So our final result is:
$$
(\bar{u}^{j+1}, \bar{\phi}_n)_h = (\bar{u}^j, \bar{\phi}_n)_h + \Delta t (A \bar{u}^j, \bar{\phi}_n)_h + \sqrt{ \frac{\Delta t}{\Delta x} } (Z^j, \bar{\phi}_n)_h \\
\boxed{
\hat{u}_n^{j+1} = (1 - \Delta t \lambda_n^{FD}) \hat{u}_n^j + \sqrt{ \frac{\Delta t}{\Delta x} } (Z^j, \bar{\phi}_n)_h
}\\
\text{This is our expression for Fourier modes. Now we'll do some analysis on the thing.}
$$

## Stationary Variance of the Discrete Fourier Mode

We'll analyse the stationary variance of $\hat{u}_n^j$ as $j \to \infty$ . The equation:

$$
\hat{u}_n^{j+1} = (1 - \Delta t \lambda_n^{FD}) \hat{u}_n^j + \sqrt{ \frac{\Delta t}{\Delta x} } (Z^j, \bar{\phi}_n)_h
$$

Define the random noise term as:

$$
\eta_n^j := \sqrt{ \frac{\Delta t}{\Delta x} } (Z^j, \bar{\phi}_n)_h = \sqrt{ \frac{\Delta t}{\Delta x} } \sum_{i=1}^N \Delta x \cdot Z_i^j \phi_n(x_i) = \sqrt{ \Delta t \Delta x } \sum_{i=1}^N Z_i^j \phi_n(x_i)
$$

Since the $Z_i^j$ are i.i.d. $\mathcal{N}(0,1)$, this is a weighted sum of independent Gaussians, so $\eta_n^j \sim \mathcal{N}(0, \sigma_n^2)$ where:

$$
\sigma_n^2 = \mathbb{V}[\eta_n^j] = \Delta t \Delta x \sum_{i=1}^N \phi_n(x_i)^2 \approx \Delta t
$$

This approximation holds because:

$$
\sum_{i=1}^N \phi_n(x_i)^2 \cdot \Delta x \approx \int_0^1 \phi_n(x)^2 dx = 1
$$

So we now have a scalar stochastic recurrence:

$$
\hat{u}_n^{j+1} = a_n \hat{u}_n^j + \eta_n^j,
\quad \text{with} \quad a_n = 1 - \Delta t \lambda_n^{FD}, \quad \eta_n^j \sim \mathcal{N}(0, \Delta t)
$$

This is a discrete-time Ornstein–Uhlenbeck process. Its stationary variance is fairly easy to derive. Just set both variances equal to each other (stationary case) and we obtain:

$$
\mathbb{V}[\hat{u}_n^\infty] = \frac{\mathbb{V}[\eta_n^j]}{1 - a_n^2}
= \frac{\Delta t}{1 - (1 - \Delta t \lambda_n^{FD})^2}
$$

Now expand the denominator:

$$
1 - (1 - \Delta t \lambda_n^{FD})^2 = 1 - \left(1 - 2 \Delta t \lambda_n^{FD} + \Delta t^2 (\lambda_n^{FD})^2 \right)
= 2 \Delta t \lambda_n^{FD} - \Delta t^2 (\lambda_n^{FD})^2
$$

So:

$$
\mathbb{V}[\hat{u}_n^\infty] = \frac{\Delta t}{2 \Delta t \lambda_n^{FD} - \Delta t^2 (\lambda_n^{FD})^2}
= \frac{1}{2 \lambda_n^{FD} - \Delta t \lambda_n^{FD^2}} = \frac{1}{2\lambda_n^{FD}(1 - \frac{\Delta t \lambda_n^{FD}}{2})}
$$

Now do a Taylor expansion of the denominator using $\frac{1}{1-x} = 1 + x + x^2 + x^3 ...$:
$$
\mathbb{V}[\hat{u}_n^\infty] = \frac{1}{2\lambda_n^{FD}}(1 + \frac{\Delta t \lambda_n^{FD}}{2} ...)\\
\text{Remembering what $\lambda_n^{FD}$ is: } \lambda_n^{FD} = \frac{4}{\Delta x^2} \sin^2\left(\frac{n\pi}{2\left(N+1\right)}\right) = \frac{4}{\Delta x^2} \sin^2\left(n \pi \Delta x \right)\\
\\
\text{Another Taylor expansion } \sin(\frac{n \pi \Delta x}{2}) = \frac{n \pi \Delta x}{2} - \frac{1}{6} (\frac{n \pi \Delta x}{2})^3 \dots\\
\\
\sin^2(n \pi \Delta x) = \frac{n^2 \pi^2 \Delta x^2}{4} + \mathcal{O}(\Delta x^4)\\
\lambda_n^{FD} =  \frac{4}{\Delta x^2} \sin^2\left( \frac{n\pi}{2\left( N+1 \right)} \right) = \frac{4}{\Delta x^2} (\frac{n^2 \pi^2 \Delta x^2}{4} + \mathcal{O}(\Delta x^4)) = n^2 \pi^2  + \mathcal{O}(\Delta x^2) = \lambda_n^2 + \mathcal{O}(\Delta x^2)
$$

Really, really close now.

So, we have that $\lambda_n^{FD} = \lambda_n^2 + \mathcal{O}(\Delta x^2)$. This is nice. We also had that $\mathbb{V}\left[\hat{u}_n^\infty\right] = \frac{1}{2\lambda_n^{FD}}\left(1 + \frac{\Delta t \lambda_n^{FD}}{2} + \dots\right)$.

Little bit of Taylor expanding and reorgaising should show that I the right result :). 

$$
\mathbb{V}\left[\hat{u}_n^\infty\right] = \frac{1}{2\lambda_n^{FD}}\left(1 + \frac{\Delta t \lambda_n^{FD}}{2} + \dots\right) \\
 = \frac{1}{2\left(\lambda_n^2 + \mathcal{O}(\Delta x^2)\right)}\left(1 + \frac{\Delta t \lambda_n^{FD}}{2} + \dots\right) \\
 = \frac{1}{2 \lambda_n^2 \left(1 + \mathcal{O}(\Delta x^2)\right)}\left(1 + \frac{\Delta t \lambda_n^{FD}}{2} + \dots\right) \\
 = \frac{1}{2 \lambda_n^2}\left(1 + \mathcal{O}(\Delta x^2)\right)\left(1 + \frac{\Delta t \lambda_n^{FD}}{2} + \dots\right) \\
 = \frac{1}{2 \lambda_n^2}\left(1 + \mathcal{O}(\Delta x^2)\right)
$$

And so we compare to the original, remember $(*)$ equation at the beginning, $\mathbb{V}\left[u_n^\infty\right] = \frac{1}{2\lambda_n^2}$

$$
|\mathbb{V}\left[u_n^\infty\right] - \mathbb{V}\left[\hat{u}_n^\infty]\right]| = |\frac{1}{2\lambda_n^2} - \frac{1}{2 \lambda_n^2}\left(1 + \mathcal{O}(\Delta x^2)\right)|
= \mathcal{O}(\Delta x^2)
$$